<img src="fifa.jpeg" style="float:left">

In [1]:
print ('auto-Commit')

auto-Commit


In [2]:
print ('manual-Commit')

manual-Commit


# Fifa 19 Dataset - Transfer Value predicting Regression Model

In [3]:
!pip install scikit-learn

     |████████████████████████████████| 22.3MB 1.0MB/s eta 0:00:01
     |████████████████████████████████| 14.8MB 52.8MB/s eta 0:00:01
     |████████████████████████████████| 25.9MB 65.1MB/s eta 0:00:01
     |████████████████████████████████| 317kB 66.8MB/s eta 0:00:01
ERROR: mosaic-connector-python 1.0.0 has requirement numpy==1.16.1, but you'll have numpy 1.19.5 which is incompatible.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
!pip install matplotlib

     |████████████████████████████████| 11.5MB 954kB/s eta 0:00:01
     |████████████████████████████████| 112kB 74.1MB/s eta 0:00:01
     |████████████████████████████████| 256kB 77.5MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/14/32/d3fa649ad7ec0b82737b92fefd3c4dd376b0bb23730715124569f38f3a08/numpy-1.19.5-cp36-cp36m-manylinux2010_x86_64.whl
     |████████████████████████████████| 1.1MB 57.8MB/s eta 0:00:01
     |████████████████████████████████| 3.1MB 60.2MB/s eta 0:00:01
ERROR: openapi-spec-validator 0.3.3 has requirement pyrsistent<0.17.0, but you'll have pyrsistent 0.17.3 which is incompatible.
ERROR: mosaic-connector-python 1.0.0 has requirement numpy==1.16.1, but you'll have numpy 1.19.5 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement matplotlib==3.1.1; python_version < "3.8", but you'll have matplotlib 3.3.4 which is incompatible.
ERROR: mosaic-ai-client 1.0.0 has requirement matplotlib==3.1.1, but you'll have matplotlib 3.3.4 wh

# Importing required Packages

In [5]:

import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from mosaicml import *
from mosaicml.constants import MLModelFlavours

import warnings
warnings.filterwarnings('ignore')

# Loading Dataset

In [6]:
# Load the data
data = pd.read_csv("/data/fifa.csv")
data.head(10)

FileNotFoundError: [Errno 2] File /data/fifa.csv does not exist: '/data/fifa.csv'

# Seperating Dataset into Training and Test

In [ ]:
X = data[['age','height_cm','weight_kg','skill_dribbling','attacking_crossing','attacking_finishing','movement_acceleration','movement_sprint_speed','power_shot_power','mentality_aggression','defending_marking','defending_standing_tackle','goalkeeping_diving','goalkeeping_handling','overall','potential']]
y = data['value_eur']
ylog = np.log(y)

%matplotlib inline
import matplotlib.pyplot as plt
plt.hist(ylog, bins='auto')
plt.title("ln(value_eur)")
plt.show()

X_train, X_test, ylog_train, ylog_test, y_train, y_test = train_test_split(X, ylog, y, test_size=0.25, random_state=4)

# Training the Model

In [ ]:
gbm_default = GradientBoostingRegressor()
gbm_default.fit(X_train, y_train)

# Defining a Score Function

In [ ]:
@scoring_func
def score(model, request):
    payload = request.json["payload"]
    data_list = payload
    data_array = np.asarray(data_list)
    try:
        prediction = model.predict(data_array)
    except:
        prediction = model.predict(data_array.reshape(1, -1))
    return prediction.tolist()


# Testing Score Function

In [ ]:
# to verify if score func works

import requests
req = requests.Request()
req.json = {"payload":X_test}

'''
Reference Sample Payload 

req.json = {"payload": [ 25, 185,  88,  78,  65,  81,  77,  75,  81,  63,  37,  26,   9,
        10,  79,  82]}
        
'''
score(gbm_default, req)

# Calculating Y_true and Y_pred

In [ ]:
Y_true = y_test

Y_pred = gbm_default.predict(X_test)
features = list(X_train)

# Registering the Model

In [ ]:
register_model(
    gbm_default,
    score,
    "fifa_model",
    "fifa_regression",
    MLModelFlavours.sklearn,
    init_script="mkdir abc \\n cd abc \\n touch abc.txt",
    input_type="json",
    explain_ai=True,
    x_train=X_train, 
    y_train=y_train, 
    y_true=Y_true,
    y_pred=Y_pred,
    feature_names=features,
    feature_ids=features,
    target_names=["Value"],
    model_type="regression"
)

In [ ]:
!pip freeze | grep numpy